Part of this project focuses on the classical R&S setting where the solution space is
discrete. In discrete spaces, we have repeat observations from the same design point,
which leads to the covariance matrix `K(X, X)` becoming singular. We need the Cholesky
factor of the covariance matrix for MLE, which doesn't exist for singular matrices.

The idea we want to explore here is whether we can decompose observations into parts
in a consistent manner. My intuition says that the resulting likelihood will depend on
the ordering of the samples, which is very much not desirable. Let's see if this is
true.

# Singularity is a numerical issue! Use double!

In [9]:
import torch
from gpytorch.kernels import MaternKernel
from torch.distributions import MultivariateNormal

base_kernel = MaternKernel()

alternatives = torch.tensor(range(3), dtype=torch.float).view(-1, 1)

full_kernel = base_kernel(alternatives).evaluate_kernel().evaluate()

In [2]:
full_kernel

tensor([[1.0000, 0.3056, 0.0336],
        [0.3056, 1.0000, 0.3056],
        [0.0336, 0.3056, 1.0000]], grad_fn=<MaternCovarianceBackward>)

Now suppose that we have 2, 3, 4 observations from each alternative respectively.

In [3]:
indices = torch.tensor([0, 0, 1, 1, 1, 2, 2, 2, 2])
X = alternatives[indices].view(-1, 1)
kernel = base_kernel(X)
kernel_eval = kernel.evaluate_kernel().evaluate()

print(f"Full kernel: {kernel_eval}")

try:
    chol = kernel_eval.cholesky()
except RuntimeError:
    chol = 'Singular matrix!'
print(f"Cholesky?: {chol}")

Full kernel: tensor([[1.0000, 1.0000, 0.3056, 0.3056, 0.3056, 0.0336, 0.0336, 0.0336, 0.0336],
        [1.0000, 1.0000, 0.3056, 0.3056, 0.3056, 0.0336, 0.0336, 0.0336, 0.0336],
        [0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 0.3056, 0.3056, 0.3056, 0.3056],
        [0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 0.3056, 0.3056, 0.3056, 0.3056],
        [0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 0.3056, 0.3056, 0.3056, 0.3056],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 1.0000],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 1.0000],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 1.0000],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 1.0000]],
       grad_fn=<MaternCovarianceBackward>)
Cholesky?: Singular matrix!


How about adding some noise to the covariance matrix?

In [32]:
noisy_kernel = kernel_eval + torch.eye(9) * 0.01

print(f"Noisy kernel: {noisy_kernel}")

try:
    chol = noisy_kernel.cholesky()
except RuntimeError:
    chol = 'Singular matrix!'
print(f"Cholesky?: {chol}")


Noisy kernel: tensor([[1.0100, 1.0000, 0.3056, 0.3056, 0.3056, 0.0336, 0.0336, 0.0336, 0.0336],
        [1.0000, 1.0100, 0.3056, 0.3056, 0.3056, 0.0336, 0.0336, 0.0336, 0.0336],
        [0.3056, 0.3056, 1.0100, 1.0000, 1.0000, 0.3056, 0.3056, 0.3056, 0.3056],
        [0.3056, 0.3056, 1.0000, 1.0100, 1.0000, 0.3056, 0.3056, 0.3056, 0.3056],
        [0.3056, 0.3056, 1.0000, 1.0000, 1.0100, 0.3056, 0.3056, 0.3056, 0.3056],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0100, 1.0000, 1.0000, 1.0000],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0100, 1.0000, 1.0000],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0000, 1.0100, 1.0000],
        [0.0336, 0.0336, 0.3056, 0.3056, 0.3056, 1.0000, 1.0000, 1.0000, 1.0100]],
       grad_fn=<AddBackward0>)
Cholesky?: tensor([[1.0050, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9950, 0.1411, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3041, 0.0215, 0.9576, 0.0000, 

What if we instead grouped the observations so that each group only includes a given
alternative only once. The resulting groups are [0, 1, 2], [0, 1, 2], [1, 2] and [2].

We can now obtain the Cholesky factor for each group as follows.

In [6]:
kernel_g1 = full_kernel
kernel_g2 = full_kernel
kernel_g3 = full_kernel[[1, 2]][:, [1, 2]]
kernel_g4 = full_kernel[2, 2].view(1, 1)

chol_g1 = full_kernel.cholesky()
chol_g2 = full_kernel.cholesky()
chol_g3 = full_kernel.cholesky()
chol_g4 = full_kernel.cholesky()

With this grouping, we could calculate the log-likelihood as the sum of log-likelihoods
 of each group.

Let's see with a simple MVN if the sum of log-probabilities of MVN samples depend on
the ordering of the observations.

In [26]:
mean = torch.tensor([0., 0.5, 1.0])
covar = torch.tensor([
    [1.0, 0.5, 0.25], [0.5, 1.0, 0.5], [0.25, 0.5, 1.0]
])
sample_mvn = MultivariateNormal(mean, covar)

samples = sample_mvn.rsample(torch.Size([4]))

In [27]:
lls = sample_mvn.log_prob(samples)
print(f"likelihoods: {lls}, total: {lls.sum()}")

likelihoods: tensor([-2.9397, -3.3755, -5.2740, -2.9170]), total: -14.506156921386719


Shuffle the samples

In [28]:
samples_copy = samples.clone()

samples_copy[0, 0], samples_copy[2, 0] = samples[2, 0], samples[0, 0]
samples_copy[1, 1], samples_copy[3, 1] = samples[3, 1], samples[1, 1]
samples_copy[1, 2], samples_copy[2, 2] = samples[2, 2], samples[1, 2]

In [29]:
lls = sample_mvn.log_prob(samples_copy)

print(f"likelihoods: {lls}, total: {lls.sum()}")


likelihoods: tensor([-4.7268, -3.5360, -3.6799, -2.9201]), total: -14.862839698791504


In [30]:
samples

tensor([[-0.4147, -0.1913,  1.2397],
        [-0.7561,  0.2859,  1.8437],
        [ 1.2930, -0.2089,  1.7020],
        [-0.8997, -0.1768,  0.7761]])

In [31]:
samples_copy

tensor([[ 1.2930, -0.1913,  1.2397],
        [-0.7561, -0.1768,  1.7020],
        [-0.4147, -0.2089,  1.8437],
        [-0.8997,  0.2859,  0.7761]])